# Spreadsheet Cleaner

This script deletes duplicate rows from a spreadsheet file. First you have to specify input and output file. The file extension will be used to determine whether it should be treated as CSV (\*.csv) or Excel (\*.xlsx) file.

In [25]:
input_file = './test.csv'
output_file = './test_clean.csv'

These files should contain comma-seperated values (CSV format). But you can also specify a different delimiter or quote-character (standard is ',' and '"')

In [34]:
delimiter=','
quotechar='"'

Now you have to specify which columns should be used to compare rows. A row will be deleted if there is another row later in the file that has identical entries in all the specified columns. Note 1: the more columns you specify the less rows will be deleted because *all* columns have to match before deleting the respective row. Note 2: if you get an error *'IndexError: list index out of range'* somewhere below this may either be because one of the specified colum indices is too large or because you specified the wrong delimiter above (only relevant for CSV input files) so that all columns are considered as only one.

In [37]:
compare_columns=[1,2]

Now happens the "magic" ;-)

*(Technical note: This script is actually very crude since for every row it iterates over all other rows. That is for *n* rows we get *O(n^2)* complexity. First sorting all rows and then deleting rows with identical successor row would result in an improvement to *O(log(n)+n)*. But speed is not an issue here and I was too lazy to implement it that way :-) )*

In [41]:
## read/write csv files
import csv
## read/write excel files
from openpyxl import load_workbook, Workbook
## display html
from IPython.core.display import HTML, display
## exceptions
import sys

## variables for input, output, html display, and counting deleted rows
spamreader = []
spamwriter = []
html = []
rows_deleted = 0

## read input file
if input_file.endswith('.csv'):
    print("Reading CSV file '{}'".format(input_file))
    with open(input_file, 'r') as csvfile_in:
        spamreader = list(csv.reader(csvfile_in, delimiter=delimiter, quotechar=quotechar))
elif input_file.endswith('.xlsx'):
    print("Reading Excel file '{}'".format(input_file))
    wb = load_workbook(input_file)
    for sheet in wb:
        print("Using sheet '{}'".format(sheet.title))
        spamreader = [[cell.value for cell in row] for row in sheet.rows]
        break
else:
    raise IOError("Unknown input format")

## iterate through rows
for idx1,row1 in enumerate(spamreader):

    ## check for duplicates by (again) iterating through rows
    found_duplicate = False
    for idx2,row2 in enumerate(spamreader):
        ## skip rows that were already checked including the one currently being checked
        if idx2 <= idx1:
            continue
        ## check if any of the specified columns is NOT equal
        all_equal = True
        for col_idx in compare_columns:
            if row1[col_idx] != row2[col_idx]:
                all_equal = False
                break
        ## if all specified colums ARE equal the two rows are considered to be a duplicate of each other
        if all_equal:
            found_duplicate = True
            duplicate_row_idx = idx2
            break

    ## update counter
    if found_duplicate:
        rows_deleted += 1

    ## add row to html output
    if found_duplicate:
        background_col = '#fdd'
        html.append('<tr style="background-color:#fdd"><td><b>{0}</b> (duplicate of {1})</td>'.format(idx1,duplicate_row_idx))
    else:
        html.append('<tr><td style="background-color:#eee"><b>{}</b></td>'.format(idx1))
        spamwriter.append(row1)
    for col_idx,cell in enumerate(row1):
        if not found_duplicate:
            if col_idx in compare_columns:
                background_col = '#ddf'
            else:
                background_col = '#fff'
        html.append('<td style="background-color:{0}">{1}</td>'.format(background_col,cell))
    html.append('</tr>')

## print number of deleted rows
print('Deleted {} out of {} rows'.format(rows_deleted,len(spamreader)))

## finish html table by adding header/footer and enclosing in <table> tag
head = []
head.append('<tr style="background-color:#eee"><th></th>')
for col_idx,col in enumerate(spamreader[0]):
    if col_idx in compare_columns:
        head.append('<th style="background-color:#ddf">{}</th>'.format(col_idx))
    else:
        head.append('<th>{}</th>'.format(col_idx))
head.append('</tr>')
html = ['<table width=100%>'] + head + html + head + ['</table>']
table = ''.join(html)

## write output file
if output_file.endswith('.csv'):
    print("Writing CSV file '{}'".format(output_file))
    with open(output_file, 'w') as csvfile_out:
        outwriter = csv.writer(csvfile_out, delimiter=delimiter, quotechar=delimiter)
        for row in spamwriter:
            outwriter.writerow(row)
elif output_file.endswith('.xlsx'):
    print("Writing Excel file '{}'".format(output_file))
    wb = Workbook()
    sheet = wb.active
    for row_idx,row in enumerate(spamwriter):
        for col_idx,cell in enumerate(row):
            ## indices start with 1 not 0
            sheet.cell(row=row_idx+1,column=col_idx+1).value = cell
    wb.save(output_file)
else:
    raise IOError("Unknown output format")

display(HTML(table))

Reading CSV file './test.csv'
Deleted 1 out of 4 rows
Writing CSV file './test_clean.csv'


,0,1,2,3,4,5
0,lsadlkj,lj,lkj,lkj,jkjk,
1 (duplicate of 3),lkj,kj,kj,kj,kj,asdf
2,,lkj,jkk,jk,jkjk,
3,lkj,kj,kj,kj,kj,asdf
,0,1,2,3,4,5
